In [16]:
#initialise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn.linear_model import LinearRegression

In [22]:
#import datasets

recruit = pd.read_csv('/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Average Monthly Recruitment:Resignation Rates by Industry and Occupational Group, Quarterly/average-monthly-recruitment-resignation-rate-by-industry-and-occupational-group-level-3.csv',
                        header=0)

vacancy = pd.read_csv('/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Job Vacancy By Industry and Occupational Group, Quarterly/job-vacancy-by-industry-and-occupational-group-level3.csv',
                        header=0)

vacancyr = pd.read_csv('/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Job Vacancy By Industry and Occupational Group, Quarterly/job-vacancy-rate-by-industry-and-occupational-group-level3.csv',
                        header=0)

# join recruit, vacancy, vacancyr dataframes
jobs = vacancy.merge(vacancyr,on=['quarter','industry1','industry2','industry3','occupation1'])[['quarter','industry1','industry2','industry3','occupation1','job_vacancy','job_vacancy_rate']].merge(recruit,on=['quarter','industry1','industry2','industry3','occupation1'])[['quarter','industry1','industry2','industry3','occupation1','job_vacancy','job_vacancy_rate','recruitment_rate','resignation_rate']]
